In [ ]:
pip install pyro-ppl

# Linear regression

A linear model on the regression will provide as a basic estimation as to the performance of the data. The fast approximation will be a rough model and lack computational complexity.

In [2]:
import os
from functools import partial
import torch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import pyro
import pyro.distributions as dist

# for CI testing
smoke_test = ('CI' in os.environ)
assert pyro.__version__.startswith('1.9.0')
pyro.set_rng_seed(1)

%matplotlib inline
plt.style.use('default')

In [3]:
from torch import nn
from pyro.nn import PyroModule

assert issubclass(PyroModule[nn.Linear], nn.Linear)
assert issubclass(PyroModule[nn.Linear], PyroModule)

In [9]:
train = pd.read_csv("/content/sample_data/california_housing_train.csv")
test = pd.read_csv("/content/sample_data/california_housing_test.csv")

df = pd.concat([train, test])

df['position'] = np.sqrt((df['longitude'] + df['latitude'])*(df['longitude'] + df['latitude']))
df['value'] = np.log(df['median_house_value'])

In [32]:
data = torch.tensor(df[["position",
                        "median_income",
                        "total_rooms",
                        "housing_median_age",
                        "value"]].values,dtype=torch.float)

In [11]:
x_data = data[:,:-1]
x_data

tensor([[8.0120e+01, 1.4936e+00, 5.6120e+03, 1.5000e+01],
        [8.0070e+01, 1.8200e+00, 7.6500e+03, 1.9000e+01],
        [8.0870e+01, 1.6509e+00, 7.2000e+02, 1.7000e+01],
        ...,
        [8.3400e+01, 2.2895e+00, 9.5600e+02, 1.0000e+01],
        [8.3020e+01, 3.2708e+00, 9.6000e+01, 4.0000e+01],
        [8.5210e+01, 8.5608e+00, 1.7650e+03, 4.2000e+01]])

In [12]:
y_data = data[:,-1]
y_data

tensor([11.1110, 11.2910, 11.3586,  ..., 11.0349, 11.9984, 13.1224])

In [13]:
linear_reg_model = PyroModule[nn.Linear](4, 1)

In [14]:
loss_fn = torch.nn.MSELoss(reduction='sum')
optim = torch.optim.Adam(linear_reg_model.parameters(), lr=0.05)
num_iterations = 1500 if not smoke_test else 2

In [15]:
pyro.clear_param_store()

def train():
    # run the model forward on the data
    y_pred = linear_reg_model(x_data).squeeze(-1)
    # calculate the mse loss
    loss = loss_fn(y_pred, y_data)
    # initialize gradients to zero
    optim.zero_grad()
    # backpropagate
    loss.backward()
    # take a gradient step
    optim.step()
    return loss

for j in range(num_iterations):
    loss = train()
    if (j + 1) % 50 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss.item()))


# Inspect learned parameters
print("Learned parameters:")
for name, param in linear_reg_model.named_parameters():
    print(name, param.data.numpy())

[iteration 0050] loss: 7018119.5000
[iteration 0100] loss: 20252.2383
[iteration 0150] loss: 15700.3359
[iteration 0200] loss: 13725.2666
[iteration 0250] loss: 12146.3945
[iteration 0300] loss: 10717.4707
[iteration 0350] loss: 9438.5732
[iteration 0400] loss: 8303.8457
[iteration 0450] loss: 7307.1387
[iteration 0500] loss: 6442.5371
[iteration 0550] loss: 5703.3154
[iteration 0600] loss: 5081.1318
[iteration 0650] loss: 4565.8877
[iteration 0700] loss: 4146.1191
[iteration 0750] loss: 3809.6484
[iteration 0800] loss: 3544.2466
[iteration 0850] loss: 3338.2104
[iteration 0900] loss: 3180.7695
[iteration 0950] loss: 3062.3398
[iteration 1000] loss: 2974.6438
[iteration 1050] loss: 2910.7195
[iteration 1100] loss: 2864.8540
[iteration 1150] loss: 2832.4636
[iteration 1200] loss: 2809.9531
[iteration 1250] loss: 2794.5593
[iteration 1300] loss: 2784.2026
[iteration 1350] loss: 2777.3481
[iteration 1400] loss: 2772.8853
[iteration 1450] loss: 2770.0283
[iteration 1500] loss: 2768.2280
Le

# Bayesian Regression with stochastic variational inference

In [16]:
from pyro.nn import PyroSample


class BayesianRegression(PyroModule):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.linear = PyroModule[nn.Linear](in_features, out_features)
        self.linear.weight = PyroSample(dist.Normal(0., 1.).expand([out_features, in_features]).to_event(2))
        self.linear.bias = PyroSample(dist.Normal(0., 10.).expand([out_features]).to_event(1))

    def forward(self, x, y=None):
        sigma = pyro.sample("sigma", dist.Uniform(0., 10.))
        mean = self.linear(x).squeeze(-1)
        with pyro.plate("data", x.shape[0]):
            obs = pyro.sample("obs", dist.Normal(mean, sigma), obs=y)
        return mean

In [17]:
from pyro.infer.autoguide import AutoDiagonalNormal

model = BayesianRegression(4, 1)
guide = AutoDiagonalNormal(model)

In [18]:
from pyro.infer import SVI, Trace_ELBO

adam = pyro.optim.Adam({"lr": 0.03})
svi = SVI(model, guide, adam, loss=Trace_ELBO())

In [19]:
pyro.clear_param_store()
for j in range(num_iterations):
    # calculate the loss and take a gradient step
    loss = svi.step(x_data, y_data)
    if j % 100 == 0:
        print("[iteration %04d] loss: %.4f" % (j + 1, loss / len(data)))

[iteration 0001] loss: 357.8314
[iteration 0101] loss: 144.7227
[iteration 0201] loss: 424.3316
[iteration 0301] loss: 156.1727
[iteration 0401] loss: 5.7386
[iteration 0501] loss: 33.6322
[iteration 0601] loss: 4.2875
[iteration 0701] loss: 6.7598
[iteration 0801] loss: 6.7397
[iteration 0901] loss: 9.5057
[iteration 1001] loss: 11.8670
[iteration 1101] loss: 28.0911
[iteration 1201] loss: 3.2732
[iteration 1301] loss: 27.0088
[iteration 1401] loss: 37.1257


In [20]:
guide.requires_grad_(False)

for name, value in pyro.get_param_store().items():
    print(name, pyro.param(name))

AutoDiagonalNormal.loc Parameter containing:
tensor([ 1.2649,  0.0878,  0.6300,  0.0044,  0.1625, -3.4021])
AutoDiagonalNormal.scale tensor([0.0705, 0.0397, 0.1241, 0.0065, 0.0470, 0.0877])


In [21]:
from pyro.infer import Predictive


def summary(samples):
    site_stats = {}
    for k, v in samples.items():
        site_stats[k] = {
            "mean": torch.mean(v, 0),
            "std": torch.std(v, 0),
            "5%": v.kthvalue(int(len(v) * 0.05), dim=0)[0],
            "95%": v.kthvalue(int(len(v) * 0.95), dim=0)[0],
        }
    return site_stats


predictive = Predictive(model, guide=guide, num_samples=800,
                        return_sites=("linear.weight", "obs", "_RETURN"))
samples = predictive(x_data)
pred_summary = summary(samples)

In [34]:
mu = pred_summary["_RETURN"]
y = pred_summary["obs"]
predictions = pd.DataFrame({
    "position": x_data[:, 0],
    "median_income": x_data[:, 1],
    "total_rooms": x_data[:, 2],
    "housing_median_age": x_data[:, 3],
    "mu_mean": mu["mean"],
    "mu_perc_5": mu["5%"],
    "mu_perc_95": mu["95%"],
    "y_mean": y["mean"],
    "y_perc_5": y["5%"],
    "y_perc_95": y["95%"],
    "true_val": y_data,
})

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(12, 6), sharey=True)

fig.suptitle("Regression line 90% CI", fontsize=16)

In [ ]:
weight = samples["linear.weight"]
weight = weight.reshape(weight.shape[0], 3)
gamma_within_africa = weight[:, 1] + weight[:, 2]
gamma_outside_africa = weight[:, 1]
fig = plt.figure(figsize=(10, 6))
sns.distplot(gamma_within_africa, kde_kws={"label": "African nations"},)
sns.distplot(gamma_outside_africa, kde_kws={"label": "Non-African nations"})
fig.suptitle("Density of Slope : log(GDP) vs. Terrain Ruggedness");